Pierwszym krokiem jest stworzenie populacji losowych ciągów bitowych. Możemy użyc wartości logicznych "True" i "False", wartości string '0' i '1' lub wartości int 0 i 1. W tym przykładzie, wykorzystamy wartości liczbowe.

Możemy wygenerować tablicę wartości całkowitych w zadanym zakresie za pomocą funkcji randint() a następnie określić zakres jako wartości zaczynąjące się od 0 a kończące na 2. Naszymi wynikami bedą zera i jedynki. Aby przedstawić przykład w jak najprostszy sposób, proponowane rozwiązanie zaprezentuję jako listę zamiast tablicy NumPy.

Poniżej prezentuję przykład utworzenia początkowej populacji losowej ciągu bitów. 'n_pop' jest hiperparametrem kontrolującym wielkość populacji, a 'n_bits' jest hiperparametrem definiującym liczbę bitów w pojedynczym rozwiązaniu.

In [1]:
# Wczytywanie używanej biblioteki
from numpy.random import randint
from numpy.random import rand

In [2]:
# Przykładowy kod tworzący populacje losowych ciągów bitowych
n_bits= 5
n_pop = 2
pop = [randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
print(pop)

[[1, 1, 0, 1, 0], [1, 1, 1, 1, 1]]


Następnie możemy wyliczyć stałą liczbę iteracji algorytmu, w tym przypadku za tą stałą będzie odpowiedzialny hiperparametr o nazwie 'n_iter'

In [21]:
# Kod w celu demonstracji- nie uruchamiać
# Do wyliczenia stałej liczby iteracji użyjemy pętli for (przykład)
 for gen in range(n_iter):

IndentationError: unexpected indent (1433582520.py, line 2)

Pierwszym krokiem w jednej iteracji algorytmu jest ocena wszystkich potencjalnych rozwiązań.

Użyjemy funkcji o nazwie cel() jako ogólnej funkcji celu i wywołamy ją, aby uzyskać wynik wartości funkcji, który zminimalizujemy.


In [8]:
# Kod w celu demonstracji- nie uruchamiać
#ocena wszystkich kandydatów w populacji
wyniki = [cel(c) for c in pop]

Następnie możemy wybrać rodziców, którzy zostaną wykorzystani do reprodukcji.

Procedurę selekcji można zaimplementować jako funkcję, która pobiera populację i zwraca jednego wybranego rodzica. Wartość k jest ustalona na 3 z domyślnym argumentem.

In [2]:
# selekcja
def selekcja(pop, wyniki, k=3):
	# pierwszy losowy wybór
	selekcja_ix = randint(len(pop))
	for ix in randint(0, len(pop), k-1):
		# sprawdzanie czy wybrany został lepszy 
		if wyniki[ix] < wyniki[selekcja_ix]:
			selekcja_ix = ix
	return pop[selekcja_ix]

Możemy wywołać tę funkcję jeden raz dla każdej pozycji w populacji, aby utworzyć listę rodziców.

In [30]:
# Kod w celu demonstracji- nie uruchamiać
# wybór rodziców
wybrani = [selekcja(pop, wyniki) for _ in range(n_pop)]

NameError: name 'wyniki' is not defined

Teraz możemy stworzyć kolejną generację

Stworzenie kolejnej generacji najpierw wymaga funkcji do wykonania krzyżowania. Ta funkcja będzie uwzględniać dwoje rodziców i współczynnik krzyżowania. Współczynnik krzyżowania to hiperparametr określający, czy krzyżowanie jest wykonywane, czy nie, a jeśli nie, elementy nadrzędne są kopiowane do następnej generacji. Ten parametr to współczynnik prawdopodobieństwa i zazwyczaj ma dużą wartość bliską 1,0.

Poniższa funkcja krzyzowanie() implementuje krzyżowanie poprzez losowanie liczby z zakresu [0,1] w celu ustalenia, czy krzyżowanie jest wykonywane, a następnie wybiera prawidłowy punkt podziału, jeśli ma zostać przeprowadzone krzyżowanie.

In [10]:
# krzyżowanie dwóch rodziców w celu wygenerowania dwojga potomków
def krzyzowanie(p1, p2, r_cross):
    # potomkowie domyślnie są kopią rodziców
	c1, c2 = p1.copy(), p2.copy()
	# sprawdzanie w celu rekombinacji
	if rand() < r_cross:
        # wybór punktu krzyżowania, który nie znajduje się na końcu ciągu
		pt = randint(1, len(p1)-2)
		# wykonanie krzyżowania
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]

Potrzebujemy również funkcji do przeprowadzenia mutacji.

Ta procedura po prostu odwraca bity z niskim prawdopodobieństwem kontrolowanym przez hiperparametr „r_mut”.

In [9]:
# operator mutacji
def mutacja(bitstring, r_mut):
	for i in range(len(bitstring)):
		# sprawdzanie w celu przeprowadzenie mutacji
		if rand() < r_mut:
			# odwrócenie bitu
			bitstring[i] = 1 - bitstring[i]

Aby utworzyć listę dzieci, które zostaną wykorzystane jako następne pokolenie, możemy zapętlić program przez listę rodziców, wywołując w razie potrzeby funkcje krzyżowania i mutacji

In [29]:
# Kod w celu demonstracji- nie uruchamiać
# stworzenie następnej generacji
dzieci = list()
for i in range(0, n_pop, 2):
	# połączenie wybranych rodziców w pary
	p1, p2 = wybrani[i], wybrani[i+1]
	# krzyżowanie i mutacja
	for c in krzyzowanie(p1, p2, r_cross):
		# mutacja
		mutacja(c, r_mut)
		# zapisz dla następnej generacji
		dzieci.append(c)

NameError: name 'wybrani' is not defined

Możemy połączyć to wszystko w funkcję o nazwie algorytm_genetyczny(), która pobiera nazwę funkcji celu oraz hiperparametry i zwraca najlepsze rozwiązanie znalezione podczas wyszukiwania.

In [5]:
# algorytm genetyczny
def algorytm_genetyczny(cel, n_bits, n_iter, n_pop, r_krzyz, r_mut):
	# początkowa populacja losowego ciągu bitowego
	pop = [randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
	# śledzenie najlepszego rozwiązania
	best, best_eval = 0, cel(pop[0])
	# wyliczanie pokoleń
	for gen in range(n_iter):
		# ocena wszystkich kandydatów w populacji
		wyniki = [cel(c) for c in pop]
		# sprawdzenie nowych najlepszych rozwiązań
		for i in range(n_pop):
			if wyniki[i] < best_eval:
				best, best_eval = pop[i], wyniki[i]
				print(">%d, nowe najlepsze rozwiązanie f(%s) = %.3f" % (gen,  pop[i], wyniki[i]))
		# wybór rodziców
		wybrani = [selekcja(pop, wyniki) for _ in range(n_pop)]
		# stworzenie nowej generacji
		dzieci = list()
		for i in range(0, n_pop, 2):
			# połączenie wybranych rodziców w pary
			p1, p2 = wybrani[i], wybrani[i+1]
			# krzyżowanie i mutacja
			for c in krzyzowanie(p1, p2, r_krzyz):
				mutacja(c, r_mut)
				# przechowywanie wyników dla następnego pokolenia
				dzieci.append(c)
		# zastąpienie populacji
		pop = dzieci
	return [best, best_eval]



OneMax problem

W tej sekcji zastosujemy algorytm genetyczny do problemu optymalizacji opartego na ciągach binarnych.

Problem nazywa się OneMax i ocenia ciąg binarny na podstawie liczby jedynek w ciągu. Na przykład ciąg bitów o długości 20 bitów będzie miał wynik 20 dla ciągu składającego się wyłącznie z jedynek.

Biorąc pod uwagę, że zaimplementowaliśmy algorytm genetyczny w celu zminimalizowania funkcji celu, możemy dodać znak ujemny do tej oceny, tak że duże wartości dodatnie staną się dużymi wartościami ujemnymi.

Poniższa funkcja onemax() jako dane wejściowe pobiera ciąg bitów wartości całkowitych i zwraca ujemną sumę wartości.

In [13]:
# funkcja celu
def onemax(x):
	return -sum(x)

Następnie możemy skonfigurować wyszukiwanie.

Wyszukiwanie będzie trwało 100 iteracji, a w naszych kandydujących rozwiązaniach użyjemy 20 bitów, co oznacza, że optymalna zgodność wyniesie -20,0.

Wielkość populacji wyniesie 100, a my zastosujemy współczynnik krzyżowania wynoszący 90 procent i współczynnik mutacji wynoszący 5 procent. Ta konfiguracja została wybrana metodą prób i błędów.

In [14]:
# ilość iteracji
n_iter = 100
# ilość bitów
n_bits = 20
# wielkość populacji
n_pop = 100
# współczynnik krzyżowania
r_cross = 0.9
# współczynnik mutacji
r_mut = 1.0 / float(n_bits)

Następnie można wywołać wyszukiwanie i znaleźć najlepszy wynik.

In [15]:

# przeprowadzenie wyszukiwania algorytmu genetycznego
best, score = algorytm_genetyczny(onemax, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Znaleziono najlepsze rozwiązanie!')
print('f(%s) = %f' % (best, score))

>0, nowe najlepsze rozwiązanie f([0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1]) = -12.000
>0, nowe najlepsze rozwiązanie f([1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1]) = -13.000
>0, nowe najlepsze rozwiązanie f([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0]) = -16.000
>1, nowe najlepsze rozwiązanie f([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -18.000
>5, nowe najlepsze rozwiązanie f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1]) = -19.000
>6, nowe najlepsze rozwiązanie f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -20.000
Znaleziono najlepsze rozwiązanie!
f([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]) = -20.000000


Uruchomienie przykładu spowoduje wyświetlenie najlepszego wyniku znalezionego po drodze, a następnie ostatecznego najlepszego rozwiązania na końcu wyszukiwania, które będzie rozwiązaniem optymalnym.

Continuous Function Optimization

Implementacja algorytmu genetycznego (AG) do optymalizacji funkcji ciągłej.

Celem algorytmu genetycznego będzie znalezienie takich wartości zmiennych x, dla których funkcja celu przyjmuje możliwie najniższą wartość, co odpowiada minimalizacji tej funkcji. Funkcja celu ma wartosci optymalne przy f(0, 0) = 0,0.

In [16]:
# funkcja celu
def cel(x):
	return x[0]**2.0 + x[1]**2.0

Chcąc przeprowadzić optymalizację za pomocą algorytmu genetycznego, najpierw musimy zdefiniować granice każdej zmiennej wejściowej. Ten fragment kodu definiuje zakresy dla zmiennych decyzyjnych, czyli przedziały, w których algorytm będzie przeszukiwał wartości zmiennych. W tym konkretnym przypadku granice są ustawione dla dwóch zmiennych decyzyjnych, z zakresem od -5 do 5.


In [21]:
# zakres
granice = [[-5.0, 5.0], [-5.0, 5.0]]

Przyjmiemy n_bits jako liczbę bitów na jedną zmienną wejściową do funkcji celu i ustawimy ją na 16 bitów. Zwiększenie liczby bitów może poprawić zdolność algorytmu genetycznego do dokładniejszego przeszukiwania przestrzeni poszukiwań, ale może też zwiększyć złożoność obliczeniową.

In [22]:
# liczba bitów na zmienną
n_bits = 16

Oznacza to, że nasz rzeczywisty ciąg bitów będzie miał (16 * 2) = 32 bity, biorąc pod uwagę dwie zmienne wejściowe.

Musimy odpowiednio zaktualizować nasz współczynnik mutacji, czyli prawdopodobieństwo, że pojedynczy bit w genotypie zostanie odwrócony (zmutowany). 

In [23]:
# współczynnik mutacji
r_mut = 1.0 / (float(n_bits) * len(granice))

Następnie musimy się upewnić, że początkowa populacja tworzy losowe ciągi bitów, które są wystarczająco duże. Ta początkowa populacja genotypów jest wykorzystywana jako punkt wyjścia dla algorytmu genetycznego. W miarę kolejnych iteracji algorytmu, genotypy w populacji będą ewoluować poprzez operacje selekcji, krzyżowania i mutacji w kierunku lepszych rozwiązań.

In [ ]:
# początkowa populacja losowego ciągu bitowego
pop = [randint(0, 2, n_bits*len(granice)).tolist() for _ in range(n_pop)]
print(pop)

Na koniec musimy rozszyfrować ciągi bitów na liczby przed oceną każdego z nich za pomocą funkcji celu.

Możemy to osiągnąć, najpierw rozszyfrowując każdy podciąg do liczby całkowitej, a następnie skalując liczbę całkowitą do żądanego zakresu. Otrzymamy wektor wartości w zakresie, który można następnie dostarczyć do funkcji celu, by umożliwić jego ocenę.

Poniższa funkcja rozszyfruj() przyjmuje granice funkcji, liczbę bitów na zmienną i ciąg bitów jako dane wejściowe i zwraca listę rozszyfrowanych wartości rzeczywistych.

In [26]:
# rozszyfruj ciąg bitów na liczby
def rozszyfruj(granice, n_bits, bitstring):
	rozszyfrowane = list()
	najwieksze = 2**n_bits
	for i in range(len(granice)):
		# wyodrębnij podciąg
		poczatek, koniec = i * n_bits, (i * n_bits)+n_bits
		podciag = bitstring[poczatek:koniec]
		# przekształć ciąg bitów na ciąg znaków
		znaki = ''.join([str(s) for s in podciag])
		# przekształć ciąg znaków na liczbę całkowitą
		calkowita = int(znaki, 2)
		# skaluj liczbę całkowitą do pożądanego zakresu
		wartosc = granice[i][0] + (calkowita/najwieksze) * (granice[i][1] - granice[i][0])
		# zapisz
		rozszyfrowane.append(wartosc)
	return rozszyfrowane

Możemy następnie wywołać to na początku pętli algorytmu w celu rozszyfrowania populacji, a następnie oceny każdego osobnika w populacji za pomocą funkcji celu.

In [27]:
...
# rozszyfrowanie populacji
rozszyfrowane = [rozszyfruj(granice, n_bits, p) for p in pop]
# ocena wszystkich kandydatów w populacji
wyniki = [cel(d) for d in rozszyfrowane]

Łącząc to razem, pełny przykład algorytmu genetycznego do optymalizacji funkcji ciągłej znajduje się poniżej.

In [8]:
# importowanie funkcji randint i rand z biblioteki numpy

from numpy.random import randint
from numpy.random import rand

# funkcja celu
def cel(x):
	return x[0]**2.0 + x[1]**2.0

# dekodowanie ciągu bitów na liczby
def rozszyfruj(granice, n_bits, bitstring):
	rozszyfrowane = list()
	najwieksze = 2**n_bits
	for i in range(len(granice)):
		# wyodrębnij podciąg
		poczatek, koniec = i * n_bits, (i * n_bits)+n_bits
		podciag = bitstring[poczatek:koniec]
		# przekształć ciąg bitów na ciąg znaków
		znaki = ''.join([str(s) for s in podciag])
		# przekształć ciąg znaków na liczbę całkowitą
		calkowita = int(znaki, 2)
		# skaluj liczbę całkowitą do pożądanego zakresu
		wartosc = granice[i][0] + (calkowita/najwieksze) * (granice[i][1] - granice[i][0])
		# zapisz
		rozszyfrowane.append(wartosc)
	return rozszyfrowane

# selekcja turniejowa
def selekcja(pop, wyniki, k=3):
	# pierwszy losowy wybór
	selekcja_ix = randint(len(pop))
	for ix in randint(0, len(pop), k-1):
		# sprawdź, czy lepsze
		if wyniki[ix] < wyniki[selekcja_ix]:
			selekcja_ix = ix
	return pop[selekcja_ix]

# krzyżowanie dwóch rodziców w celu wygenerowania dwojga potomków
def krzyzowanie(p1, p2, r_cross):
    # potomkowie domyślnie są kopią rodziców
	c1, c2 = p1.copy(), p2.copy()
	# sprawdzanie w celu rekombinacji
	if rand() < r_cross:
        # wybór punktu krzyżowania, który nie znajduje się na końcu ciągu
		pt = randint(1, len(p1)-2)
		# wykonanie krzyżowania
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]

# operator mutacji
def mutacja(bitstring, r_mut):
	for i in range(len(bitstring)):
		# sprawdzanie w celu przeprowadzenie mutacji
		if rand() < r_mut:
			# odwrócenie bitu
			bitstring[i] = 1 - bitstring[i]

# algorytm genetyczny
def algorytm_genetyczny(cel, granice, n_bits, n_iter, n_pop, r_cross, r_mut):
	# początkowa populacja losowego ciągu bitowego
	pop = [randint(0, 2, n_bits*len(granice)).tolist() for _ in range(n_pop)]
	# śledzenie najlepszego rozwiązania
	best, best_eval = 0, cel(rozszyfruj(granice, n_bits, pop[0]))
	# wyliczanie pokoleń
	for gen in range(n_iter):
		# rozszyfrowanie populacji
		rozszyfrowane = [rozszyfruj(granice, n_bits, p) for p in pop]
		# ocena wszystkich kandydatów w populacji
		wyniki = [cel(d) for d in rozszyfrowane]
		# sprawdzenie nowych najlepszych rozwiązań
		for i in range(n_pop):
			if wyniki[i] < best_eval:
				best, best_eval = pop[i], wyniki[i]
				print(">%d, nowe najlepsze rozwiązanie f(%s) = %f" % (gen,  rozszyfrowane[i], wyniki[i]))
		# wybór rodziców
		wybrani = [selekcja(pop, wyniki) for _ in range(n_pop)]
		# stworzenie nowej generacji
		dzieci = list()
		for i in range(0, n_pop, 2):
			# połączenie wybranych rodziców w pary
			p1, p2 = wybrani[i], wybrani[i+1]
			# krzyżowanie i mutacja
			for c in krzyzowanie(p1, p2, r_cross):
				# mutacja
				mutacja(c, r_mut)
				# przechowywanie wyników dla następnego pokolenia
				dzieci.append(c)
		# zastąpienie populacji
		pop = dzieci
	return [best, best_eval]

# zakres
granice = [[-5.0, 5.0], [-5.0, 5.0]]
# ilość iteracji
n_iter = 100
# liczba bitów na zmienną
n_bits = 16
# wielkość populacji
n_pop = 100
# współczynnik krzyżowania
r_cross = 0.9
# współczynnik mutacji
r_mut = 1.0 / (float(n_bits) * len(granice))
# przeprowadzenie wyszukiwania algorytmu genetycznego
best, score = algorytm_genetyczny(cel, granice, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Koniec!')
rozszyfrowane = rozszyfruj(granice, n_bits, best)
print('f(%s) = %f' % (rozszyfrowane, score))

>0, nowe najlepsze rozwiązanie f([-0.829925537109375, -0.375823974609375]) = 0.830020
>0, nowe najlepsze rozwiązanie f([-0.2349853515625, -0.31219482421875]) = 0.152684
>1, nowe najlepsze rozwiązanie f([-0.2349853515625, -0.296630859375]) = 0.143208
>2, nowe najlepsze rozwiązanie f([0.34271240234375, 0.0152587890625]) = 0.117685
>2, nowe najlepsze rozwiązanie f([0.082550048828125, -0.140380859375]) = 0.026521
>3, nowe najlepsze rozwiązanie f([-0.0311279296875, -0.140380859375]) = 0.020676
>3, nowe najlepsze rozwiązanie f([-0.059814453125, -0.0982666015625]) = 0.013234
>5, nowe najlepsze rozwiązanie f([0.015869140625, -0.081024169921875]) = 0.006817
>5, nowe najlepsze rozwiązanie f([0.033111572265625, -0.043182373046875]) = 0.002961
>7, nowe najlepsze rozwiązanie f([0.0006103515625, -0.042724609375]) = 0.001826
>8, nowe najlepsze rozwiązanie f([0.0006103515625, -0.04241943359375]) = 0.001800
>8, nowe najlepsze rozwiązanie f([0.0006103515625, -0.04180908203125]) = 0.001748
>10, nowe najl